# Enhanced Bitcoin Price Prediction using LSTM
## Features: Interactive Dashboard, Sentiment Analysis, Risk Analysis & Advanced Visualizations

## 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

## 2. Data Loading & Preprocessing

In [2]:
# Download Bitcoin data
btc = yf.download('BTC-USD', start='2023-01-01', end='2024-01-01')
btc.reset_index(inplace=True)
maindf = btc.copy()

# Flatten MultiIndex columns
maindf.columns = maindf.columns.droplevel(1) if hasattr(maindf.columns, 'droplevel') else maindf.columns

print(f'Dataset shape: {maindf.shape}')
print(f'Date range: {maindf["Date"].min()} to {maindf["Date"].max()}')

[*********************100%***********************]  1 of 1 completed

Dataset shape: (365, 6)
Date range: 2023-01-01 00:00:00 to 2023-12-31 00:00:00


## 3. Sentiment Analysis Integration

In [3]:
# Simulate market sentiment based on price movements and volume
def calculate_sentiment(df):
    # Price momentum sentiment
    df['Price_Change'] = df['Close'].pct_change()
    df['Volume_MA'] = df['Volume'].rolling(7).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']
    
    # Sentiment score based on price change and volume
    df['Sentiment'] = np.tanh(df['Price_Change'] * 10) * np.tanh(df['Volume_Ratio'] - 1)
    
    # Add some realistic noise
    np.random.seed(42)
    noise = np.random.normal(0, 0.1, len(df))
    df['Sentiment'] = np.clip(df['Sentiment'] + noise, -1, 1)
    
    return df

maindf = calculate_sentiment(maindf)
print(f'Sentiment range: {maindf["Sentiment"].min():.3f} to {maindf["Sentiment"].max():.3f}')

Sentiment range: -0.591 to 0.568


## 4. Risk Analysis Functions

In [4]:
def calculate_risk_metrics(prices):
    returns = prices.pct_change().dropna()
    
    # Value at Risk (95% confidence)
    var_95 = np.percentile(returns, 5)
    
    # Expected Shortfall (Conditional VaR)
    es_95 = returns[returns <= var_95].mean()
    
    # Volatility (annualized)
    volatility = returns.std() * np.sqrt(365)
    
    # Sharpe Ratio (assuming 2% risk-free rate)
    sharpe = (returns.mean() * 365 - 0.02) / volatility
    
    # Maximum Drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'VaR_95': var_95,
        'Expected_Shortfall': es_95,
        'Volatility': volatility,
        'Sharpe_Ratio': sharpe,
        'Max_Drawdown': max_drawdown,
        'Returns': returns
    }

risk_metrics = calculate_risk_metrics(maindf['Close'])
print("Risk Metrics:")
for key, value in risk_metrics.items():
    if key != 'Returns':
        print(f"{key}: {value:.4f}")

Risk Metrics:
VaR_95: -0.0306
Expected_Shortfall: -0.0441
Volatility: 0.4383
Sharpe_Ratio: 2.3074
Max_Drawdown: -0.2006


## 5. LSTM Model Training

In [5]:
# Prepare data for LSTM
closedf = maindf[['Date', 'Close']].copy()
close_data = closedf['Close'].values.reshape(-1, 1)

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_data)

# Create sequences
def create_dataset(dataset, time_step=15):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        dataX.append(dataset[i:(i + time_step), 0])
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

time_step = 15
training_size = int(len(scaled_data) * 0.60)
train_data = scaled_data[0:training_size, :]
test_data = scaled_data[training_size:len(scaled_data), :]

X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f'Training data shape: {X_train.shape}')
print(f'Test data shape: {X_test.shape}')

Training data shape: (203, 15, 1)
Test data shape: (130, 15, 1)


In [6]:
# Build and train LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                   epochs=50, batch_size=32, verbose=1)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Transform back to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_orig = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0626 - val_loss: 0.0071
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0143 - val_loss: 0.0239
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0079 - val_loss: 0.0476
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0057 - val_loss: 0.0154
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0052 - val_loss: 0.0191
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0046 - val_loss: 0.0294
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0049 - val_loss: 0.0173
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0040 - val_loss: 0.0165
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0037 - val_loss: 0.0189
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0036 - val_loss: 0.0114
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0033 - val_loss: 0.0149
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0034 - val_loss: 0.0131
E

## 6. Future Predictions

In [7]:
# Predict next 30 days
x_input = test_data[len(test_data) - time_step:].reshape(1, -1)
temp_input = list(x_input[0])
lst_output = []
pred_days = 30

for i in range(pred_days):
    if len(temp_input) > time_step:
        x_input = np.array(temp_input[1:])
        x_input = x_input.reshape(1, -1)
        x_input = x_input.reshape((1, time_step, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
    else:
        x_input = x_input.reshape((1, time_step, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())

future_predictions = scaler.inverse_transform(np.array(lst_output).reshape(-1, 1))
print(f'Generated {len(future_predictions)} future predictions')

Generated 30 future predictions


## 7. Advanced Visualizations

In [8]:
# Candlestick Chart with Predictions
fig_candle = go.Figure()

# Add candlestick
fig_candle.add_trace(go.Candlestick(
    x=maindf['Date'],
    open=maindf['Open'],
    high=maindf['High'],
    low=maindf['Low'],
    close=maindf['Close'],
    name='BTC Price'
))

# Add future predictions
future_dates = pd.date_range(start=maindf['Date'].iloc[-1] + pd.Timedelta(days=1), 
                           periods=pred_days, freq='D')
fig_candle.add_trace(go.Scatter(
    x=future_dates,
    y=future_predictions.flatten(),
    mode='lines+markers',
    name='Future Predictions',
    line=dict(color='red', width=3)
))

fig_candle.update_layout(
    title='Bitcoin Price with Future Predictions',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_dark',
    height=600
)

fig_candle.show()

## 8. Interactive Dashboard

In [9]:
# Create comprehensive dashboard
fig_dashboard = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Price Prediction vs Actual', 'Sentiment Analysis',
        'Volume Analysis', 'Risk Metrics',
        'Model Performance', 'Returns Distribution'
    ),
    specs=[
        [{"secondary_y": True}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}]
    ],
    vertical_spacing=0.08
)

# 1. Price Prediction vs Actual
test_dates = maindf['Date'].iloc[training_size + time_step + 1:training_size + time_step + 1 + len(test_predict)]
fig_dashboard.add_trace(
    go.Scatter(x=test_dates, y=y_test_orig.flatten(), name='Actual', line=dict(color='blue')),
    row=1, col=1
)
fig_dashboard.add_trace(
    go.Scatter(x=test_dates, y=test_predict.flatten(), name='Predicted', line=dict(color='red')),
    row=1, col=1
)

# 2. Sentiment Analysis
fig_dashboard.add_trace(
    go.Scatter(x=maindf['Date'], y=maindf['Sentiment'], name='Sentiment', 
              line=dict(color='green'), fill='tonexty'),
    row=1, col=2
)

# 3. Volume Analysis
fig_dashboard.add_trace(
    go.Bar(x=maindf['Date'], y=maindf['Volume'], name='Volume', 
           marker_color='purple', opacity=0.7),
    row=2, col=1
)

# 4. Risk Metrics (as text)
risk_text = f"""VaR (95%): {risk_metrics['VaR_95']:.4f}<br>
Expected Shortfall: {risk_metrics['Expected_Shortfall']:.4f}<br>
Volatility: {risk_metrics['Volatility']:.4f}<br>
Sharpe Ratio: {risk_metrics['Sharpe_Ratio']:.4f}<br>
Max Drawdown: {risk_metrics['Max_Drawdown']:.4f}"""

fig_dashboard.add_annotation(
    text=risk_text,
    xref="x4", yref="y4",
    x=0.5, y=0.5,
    showarrow=False,
    font=dict(size=12),
    align="left"
)

# 5. Model Performance (Training Loss)
fig_dashboard.add_trace(
    go.Scatter(y=history.history['loss'], name='Training Loss', line=dict(color='orange')),
    row=3, col=1
)
fig_dashboard.add_trace(
    go.Scatter(y=history.history['val_loss'], name='Validation Loss', line=dict(color='red')),
    row=3, col=1
)

# 6. Returns Distribution
fig_dashboard.add_trace(
    go.Histogram(x=risk_metrics['Returns'], name='Returns Distribution', 
                marker_color='cyan', opacity=0.7),
    row=3, col=2
)

# Update layout
fig_dashboard.update_layout(
    title_text="Bitcoin Trading Dashboard",
    template='plotly_dark',
    height=1000,
    showlegend=True
)

fig_dashboard.show()

## 9. 3D Visualization

In [10]:
# 3D Price, Volume, Sentiment visualization
fig_3d = go.Figure(data=[go.Scatter3d(
    x=maindf['Close'],
    y=maindf['Volume'],
    z=maindf['Sentiment'],
    mode='markers+lines',
    marker=dict(
        size=5,
        color=maindf['Close'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Price")
    ),
    line=dict(color='white', width=2),
    text=maindf['Date'].dt.strftime('%Y-%m-%d'),
    hovertemplate='<b>Date:</b> %{text}<br>' +
                  '<b>Price:</b> $%{x:,.2f}<br>' +
                  '<b>Volume:</b> %{y:,.0f}<br>' +
                  '<b>Sentiment:</b> %{z:.3f}<extra></extra>'
)])

fig_3d.update_layout(
    title='3D Bitcoin Analysis: Price vs Volume vs Sentiment',
    scene=dict(
        xaxis_title='Price (USD)',
        yaxis_title='Volume',
        zaxis_title='Sentiment Score',
        bgcolor='black'
    ),
    template='plotly_dark',
    height=700
)

fig_3d.show()

## 10. Model Evaluation & Summary

In [11]:
# Calculate model performance metrics
train_rmse = math.sqrt(mean_squared_error(y_train_orig, train_predict))
test_rmse = math.sqrt(mean_squared_error(y_test_orig, test_predict))
train_r2 = r2_score(y_train_orig, train_predict)
test_r2 = r2_score(y_test_orig, test_predict)

print("=" * 60)
print("ENHANCED BITCOIN PREDICTION MODEL SUMMARY")
print("=" * 60)
print(f"Training RMSE: ${train_rmse:,.2f}")
print(f"Testing RMSE: ${test_rmse:,.2f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Testing R²: {test_r2:.4f}")
print("\nRisk Metrics:")
print(f"Value at Risk (95%): {risk_metrics['VaR_95']:.4f}")
print(f"Sharpe Ratio: {risk_metrics['Sharpe_Ratio']:.4f}")
print(f"Maximum Drawdown: {risk_metrics['Max_Drawdown']:.4f}")
print("\nPredictions:")
print(f"Current Price: ${maindf['Close'].iloc[-1]:,.2f}")
print(f"Next Day Prediction: ${future_predictions[0][0]:,.2f}")
print(f"30-Day Average Prediction: ${future_predictions.mean():,.2f}")
print(f"Average Sentiment: {maindf['Sentiment'].mean():.3f}")
print("=" * 60)

ENHANCED BITCOIN PREDICTION MODEL SUMMARY
Training RMSE: $1,102.60
Testing RMSE: $1,768.62
Training R²: 0.8557
Testing R²: 0.9266

Risk Metrics:
Value at Risk (95%): -0.0306
Sharpe Ratio: 2.3074
Maximum Drawdown: -0.2006

Predictions:
Current Price: $42,265.19
Next Day Prediction: $41,255.06
30-Day Average Prediction: $37,302.40
Average Sentiment: 0.011


## 11. Interactive Prediction Plot

In [12]:
# Create final interactive prediction plot
fig_final = go.Figure()

# Historical prices
fig_final.add_trace(go.Scatter(
    x=maindf['Date'],
    y=maindf['Close'],
    mode='lines',
    name='Historical Price',
    line=dict(color='blue', width=2)
))

# Test predictions
fig_final.add_trace(go.Scatter(
    x=test_dates,
    y=test_predict.flatten(),
    mode='lines',
    name='Model Predictions',
    line=dict(color='orange', width=2)
))

# Future predictions
fig_final.add_trace(go.Scatter(
    x=future_dates,
    y=future_predictions.flatten(),
    mode='lines+markers',
    name='Future Predictions',
    line=dict(color='red', width=3, dash='dash'),
    marker=dict(size=6)
))

# Add confidence bands for future predictions
std_dev = np.std(test_predict - y_test_orig)
upper_bound = future_predictions.flatten() + 2 * std_dev
lower_bound = future_predictions.flatten() - 2 * std_dev

fig_final.add_trace(go.Scatter(
    x=future_dates,
    y=upper_bound,
    fill=None,
    mode='lines',
    line_color='rgba(0,0,0,0)',
    showlegend=False
))

fig_final.add_trace(go.Scatter(
    x=future_dates,
    y=lower_bound,
    fill='tonexty',
    mode='lines',
    line_color='rgba(0,0,0,0)',
    name='Confidence Interval',
    fillcolor='rgba(255,0,0,0.2)'
))

fig_final.update_layout(
    title='Enhanced Bitcoin Price Prediction with Confidence Intervals',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600,
    hovermode='x unified'
)

fig_final.show()

print("\n🚀 Enhanced Bitcoin Prediction Model Complete!")
print("Features included:")
print("✅ Interactive Dashboard")
print("✅ Sentiment Analysis")
print("✅ Risk Analysis (VaR, Sharpe Ratio, etc.)")
print("✅ Advanced Visualizations (3D, Candlestick, etc.)")
print("✅ Confidence Intervals")
print("✅ Comprehensive Model Evaluation")


🚀 Enhanced Bitcoin Prediction Model Complete!
Features included:
✅ Interactive Dashboard
✅ Sentiment Analysis
✅ Risk Analysis (VaR, Sharpe Ratio, etc.)
✅ Advanced Visualizations (3D, Candlestick, etc.)
✅ Confidence Intervals
✅ Comprehensive Model Evaluation
